In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import math

In [2]:
Station_list = ["POLESOUT", "KISSR0.0", "LZ2", "S133", "TCNS228", "S135",
                "FEBIN", "MBOXSOU", "MH24000", "FEBOUT", "L005", "L008", "LZ40", "L004", "S308C",
                "PALMOUNT", "S169", "S236", "POLE3S", "RITTAE2", "LZ2FA", "L007", "PELBAY3", "L006", "LZ30"]
Variable_list = ['NITRATE+NITRITE-N', 'NITRITE-N', 'AMMONIA-N', 'KJELDAHL NITROGEN, TOTAL', 'PHOSPHATE, ORTHO AS P',
                 'PHOSPHATE, TOTAL AS P', 'NITRATE-N', 'SILICA', 'CARBON, TOTAL ORGANIC', 'CARBON, DISSOLVED ORGANIC',
                 'TOTAL NITROGEN', 'PHOSPHATE, DISSOLVED AS P', 'KJELDAHL NITROGEN, DIS', 'CARBON, TOTAL', 
                 'CARBON, TOTAL INORGANIC', 'NITROGEN, TOTAL DISSOLVED',
                 'CHLOROPHYLL-A', 'PHEOPHYTIN', 'CHLOROPHYLL-A, CORRECTED', 
                 'CHLOROPHYLL-C', 'CAROTENOIDS', 'CHLOROPHYLL-B', 'CHLOROPHYLL-A(LC)', 
                 'PHEOPHYTIN-A(LC)', 'CHLOROPHYLL-B(LC)', 'RESP. PLANKTONIC']

In [3]:
def Clean_date(df):
    for index, value in enumerate(df["Collection_Date"]):
        df["Collection_Date"][index] = datetime.strptime(value, '%Y-%m-%d %H:%M:%S')        
    return df

Double check the unit!

In [4]:
def Data_clean_up(data):
    """
    The columns of the final table:
    - Time (month-year)
    - Value of variable 1
    .
    .
    - Value of variable n
    """
    #clean date: 
    data = Clean_date(data)
    
    Time_array = []
    for year in range(2000, 2020):
        for month in range(1,13):
            Time_array.append(datetime(year, month, 1))
    
    clean_data = pd.DataFrame({"MonthYear": Time_array})
    
    global Variable_list
    for variable in Variable_list:
        Value_array = []
        for time in Time_array:
            #select rows based on the given date and variable:
            index_given_time = [True if (x.month == time.month and x.year == time.year) else False for x in data["Collection_Date"]]
            index_given_variable = data["Test Name"] == variable
            #combine 2 criteria:
            index_to_choose = index_given_time & index_given_variable
            
            values = data.loc[index_to_choose, "Value"]
            #Filter negative and NA values:
            values = [x for x in values if (not math.isnan(x)) and x >= 0]
         
            if len(values) > 0:
                Value_array.append(np.mean(values))
                
            else:
                Value_array.append(None)
                
        clean_data[variable] = Value_array
        
    return clean_data

In [5]:
station = "POLESOUT"
data = pd.read_csv(station+".csv")
data = Data_clean_up(data)
data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,MonthYear,NITRATE+NITRITE-N,NITRITE-N,AMMONIA-N,"KJELDAHL NITROGEN, TOTAL","PHOSPHATE, ORTHO AS P","PHOSPHATE, TOTAL AS P",NITRATE-N,SILICA,"CARBON, TOTAL ORGANIC",...,CHLOROPHYLL-A,PHEOPHYTIN,"CHLOROPHYLL-A, CORRECTED",CHLOROPHYLL-C,CAROTENOIDS,CHLOROPHYLL-B,CHLOROPHYLL-A(LC),PHEOPHYTIN-A(LC),CHLOROPHYLL-B(LC),RESP. PLANKTONIC
0,2000-01-01,0.324,NaN,0.010,0.999,0.046,0.116,NaN,NaN,None,...,20.2,3.7,17.5,4.4,10.9,2.3,NaN,NaN,NaN,None
1,2000-02-01,0.437,NaN,NaN,1.570,0.044,0.167,NaN,10.486,None,...,7.2,NaN,6.6,1.0,5.0,NaN,NaN,NaN,NaN,None
2,2000-03-01,0.301,NaN,NaN,1.218,0.046,0.130,NaN,NaN,None,...,32.2,NaN,30.8,1.9,12.5,2.1,NaN,NaN,NaN,None
3,2000-04-01,0.007,NaN,NaN,2.512,0.008,0.184,NaN,NaN,None,...,6.2,NaN,5.5,NaN,3.0,NaN,NaN,NaN,NaN,None
4,2000-05-01,0.351,0.004,NaN,1.398,0.020,0.116,0.347,11.242,None,...,28.3,8.8,22.0,NaN,12.6,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2019-08-01,NaN,NaN,0.014,NaN,NaN,0.048,NaN,4.060,None,...,NaN,NaN,NaN,NaN,NaN,NaN,74.7,0.565,NaN,None
236,2019-09-01,NaN,0.002,0.028,NaN,0.002,0.101,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,59.6,1.450,NaN,None
237,2019-10-01,NaN,0.002,0.019,NaN,0.010,0.152,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,28.9,2.640,NaN,None
238,2019-11-01,0.012,NaN,0.014,NaN,0.002,0.046,0.012,4.220,None,...,NaN,NaN,NaN,NaN,NaN,NaN,19.5,0.587,NaN,None


Linear Regression for POLESOUT station

In [8]:
from sklearn.linear_model import LinearRegression

#Indentify indenpdent variables (predictors):
predictors = ["NITRATE+NITRITE-N", "NITRITE-N", "AMMONIA-N", "PHOSPHATE, ORTHO AS P", "PHOSPHATE, TOTAL AS P"]

index_to_choose = [True for x in range(len(data))]
#Identify the datapoints have NAs value to exclude (False value in index_to_choose)
for variable in predictors + ["CHLOROPHYLL-A(LC)"]:
    for index, value in enumerate(data[variable]):
        if pd.isna(value):
            index_to_choose[index] = False
            

X = data.loc[index_to_choose, predictors]
y = data.loc[index_to_choose, "CHLOROPHYLL-A(LC)"]
regression = LinearRegression().fit(X,y)
regression.coef_

array([  -68.94236068, -1241.87155895,  -348.96933612,  -120.27364986,
         128.65057847])